## VIVEK BHAVSHAR
## HOME DEPOT Product Search Relevence
## NLTK - Regression Model

In [2]:
import numpy as np
import pandas as pd
from nltk.stem.porter import *
stemmer = PorterStemmer()
import re
import random

random.seed(2018)

In [3]:
df_train = pd.read_csv('C:/Users/VIVEK/Downloads/train_new.csv')
df_test = pd.read_csv('C:/Users/VIVEK/Downloads/test_new.csv')
df_pro_desc = pd.read_csv('C:/Users/VIVEK/Downloads/product_descriptions_new.csv')
df_attr = pd.read_csv('C:/Users/VIVEK/Downloads/attributes_new.csv') 

In [4]:
num_train = df_train.shape[0]
num_train

74067

In [5]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True,sort=False)

In [6]:
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

In [7]:
df_all.head()

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...


## Stemming of Input Data

In [8]:
def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = re.sub(r"([0-9]+)( *)(°|degrees|degree)\.?", r"\1 deg. ", s)
        s = re.sub(r"([0-9]+)( *)(v|volts|volt)\.?", r"\1 volt. ", s)
        s = re.sub(r"([0-9]+)( *)(wattage|watts|watt)\.?", r"\1 watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1 amp. ", s)
        s = re.sub(r"([0-9]+)( *)(qquart|quart)\.?", r"\1 qt. ", s)
        s = re.sub(r"([0-9]+)( *)(hours|hour|hrs.)\.?", r"\1 hr ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per minute|gallon per minute|gal per minute|gallons/min.|gallons/min)\.?", r"\1 gal. per min. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per hour|gallon per hour|gal per hour|gallons/hour|gallons/hr)\.?", r"\1 gal. per hr ", s)
        # Deal with special characters
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("&nbsp;"," ")
        s = s.replace("&amp;","&")
        s = s.replace("&#39;","'")
        s = s.replace("/>/Agt/>","")
        s = s.replace("</a<gt/","")
        s = s.replace("gt/>","")
        s = s.replace("/>","")
        s = s.replace("<br","")
        s = s.replace("<.+?>","")
        s = s.replace("[ &<>)(_,;:!?\+^~@#\$]+"," ")
        s = s.replace("'s\\b","")
        s = s.replace("[']+","")
        s = s.replace("[\"]+","")
        s = s.replace("-"," ")
        s = s.replace("+"," ")
        # Remove text between paranthesis/brackets)
        s = s.replace("[ ]?[[(].+?[])]","")
        # remove sizes
        s = s.replace("size: .+$","")
        s = s.replace("size [0-9]+[.]?[0-9]+\\b","")
        
        
        return " ".join([stemmer.stem(re.sub('[^A-Za-z0-9-./]', ' ', word)) for word in s.lower().split()])
    else:
        return "null"

In [9]:
def str_common_word(str1, str2):
    str1, str2 = str1.lower(), str2.lower()
    words, count = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            count+=1
    return count

In [10]:
def str_whole_word(str1, str2, i_):
    str1, str2 = str1.lower().strip(), str2.lower().strip()
    count = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return count
        else:
            count += 1
            i_ += len(str1)
    return count

In [11]:
df_all.isnull().sum()

id                          0
product_uid                 0
product_title               0
search_term                 0
relevance              112067
product_description         0
dtype: int64

In [12]:
df_all['product_title'] = df_all['product_title'].apply(str_stem)
df_all['search_term'] = df_all['search_term'].apply(str_stem)
df_all['product_description'] = df_all['product_description'].apply(str_stem)

In [13]:
df_all.head() #after stemming.

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,not onli do angl make joint stronger they als...
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,not onli do angl make joint stronger they als...
2,9,100002,behr premium textur deckov 1 gal. sc 141 tugb...,deck over,3.00,behr premium textur deckov is an innov solid c...
3,16,100005,delta vero 1 handl shower onli faucet trim kit...,rain shower head,2.33,updat your bathroom with the delta vero singl ...
4,17,100005,delta vero 1 handl shower onli faucet trim kit...,shower onli faucet,2.67,updat your bathroom with the delta vero singl ...


## Feature Generation for future calculation

In [14]:
df_all['len_of_search_term'] = df_all['search_term'].apply(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].apply(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].apply(lambda x:len(x.split())).astype(np.int64)

# Create a new column that combine "search_term", "product_title" and "product_description"
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
# Number of times the entire search term appears in product title. 
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
# Number of times the entire search term appears in product description
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
# Number of words that appear in search term also appear in product title.
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
# Number of words that appear in search term also appear in production description.
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [15]:
df_all.head(2)

,id,product_uid,product_title,search_term,relevance,product_description,len_of_search_term,len_of_title,len_of_description,product_info,query_in_title,query_in_description,word_in_title,word_in_description
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.0,not onli do angl make joint stronger they als...,2,6,133,angl bracket\tsimpson strong tie 12 gaug angl\...,0,0,1,1
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.5,not onli do angl make joint stronger they als...,2,6,133,l bracket\tsimpson strong tie 12 gaug angl\tno...,0,0,1,1


In [16]:
df_all.isnull().sum()

id                           0
product_uid                  0
product_title                0
search_term                  0
relevance               112067
product_description          0
len_of_search_term           0
len_of_title                 0
len_of_description           0
product_info                 0
query_in_title               0
query_in_description         0
word_in_title                0
word_in_description          0
dtype: int64

In [17]:
df_all.columns

Index(['id', 'product_uid', 'product_title', 'search_term', 'relevance',
       'product_description', 'len_of_search_term', 'len_of_title',
       'len_of_description', 'product_info', 'query_in_title',
       'query_in_description', 'word_in_title', 'word_in_description'],
      dtype='object')

In [18]:
df_all.drop(['product_title', 'search_term', 'product_description', 'product_info'], axis=1, inplace=True)

In [19]:
df_all.head()

,id,product_uid,relevance,len_of_search_term,len_of_title,len_of_description,query_in_title,query_in_description,word_in_title,word_in_description
0,2,100001,3.00,2,6,133,0,0,1,1
1,3,100001,2.50,2,6,133,0,0,1,1
2,9,100002,3.00,2,13,170,0,0,1,1
3,16,100005,2.33,3,14,109,0,0,1,1
4,17,100005,2.67,3,14,109,1,0,3,2


In [20]:
DataFrame_train = df_all.iloc[:num_train]

In [21]:
DataFrame_train.head()

,id,product_uid,relevance,len_of_search_term,len_of_title,len_of_description,query_in_title,query_in_description,word_in_title,word_in_description
0,2,100001,3.00,2,6,133,0,0,1,1
1,3,100001,2.50,2,6,133,0,0,1,1
2,9,100002,3.00,2,13,170,0,0,1,1
3,16,100005,2.33,3,14,109,0,0,1,1
4,17,100005,2.67,3,14,109,1,0,3,2


In [22]:
DataFrame_test = df_all.iloc[num_train:]

In [23]:
DataFrame_test.head()

,id,product_uid,relevance,len_of_search_term,len_of_title,len_of_description,query_in_title,query_in_description,word_in_title,word_in_description
74067,4,100001,NaN,3,6,133,0,0,1,1
74068,5,100001,NaN,3,6,133,0,0,1,1
74069,6,100001,NaN,3,6,133,1,1,3,3
74070,7,100001,NaN,4,6,133,0,0,3,3
74071,10,100003,NaN,4,19,95,1,0,4,2


In [24]:
train_id = df_train['id']

In [25]:
test_id = df_test['id']

In [26]:
train_Y = DataFrame_train['relevance'].values

In [27]:
train_Y

array([3.  , 2.5 , 3.  , ..., 2.33, 3.  , 2.33])

In [28]:
train_X = DataFrame_train.drop(['id','relevance'],axis=1).values

In [29]:
train_X

array([[100001,      2,      6, ...,      0,      1,      1],
       [100001,      2,      6, ...,      0,      1,      1],
       [100002,      2,     13, ...,      0,      1,      1],
       ...,
       [206641,      7,     12, ...,      0,      2,      4],
       [206648,      3,      9, ...,      0,      2,      2],
       [206650,      4,     17, ...,      0,      3,      2]], dtype=int64)

In [30]:
test_X = DataFrame_test.drop(['id','relevance'],axis=1).values

In [31]:
test_X

array([[100001,      3,      6, ...,      0,      1,      1],
       [100001,      3,      6, ...,      0,      1,      1],
       [100001,      3,      6, ...,      1,      3,      3],
       ...,
       [206646,      6,     10, ...,      0,      6,      4],
       [206647,      5,      7, ...,      0,      3,      2],
       [206649,      3,     11, ...,      0,      2,      1]], dtype=int64)

## RANDOM FOREST

In [71]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=0)
rf.fit(train_X, train_Y)
y_pred = rf.predict(test_X)
y_pred

array([2.32618032, 2.32618032, 2.79949778, ..., 2.52446176, 2.28926268,
       2.10906128])

In [73]:
pd.DataFrame({"id": test_id, "relevance": y_pred}).to_csv('randomforest.csv',index=False)

## Ridge Regression

In [75]:
from sklearn.linear_model import Ridge

rg= Ridge(alpha=.1)
rg.fit(train_X, train_Y)
y_pred = rg.predict(test_X)
y_pred
#rg_mse = mean_squared_error(y_pred, y_test)
#rg_rmse = np.sqrt(rg_mse)
#print('Ridge RMSE: %.4f' % rg_rmse)

array([2.28568127, 2.28568127, 2.86997406, ..., 2.61986777, 2.15565944,
       2.17285716])

In [76]:
pd.DataFrame({"id": test_id, "relevance": y_pred}).to_csv('RidgeRegression.csv',index=False)

## Gradient Boosting for regression

In [77]:
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls').fit(train_X, train_Y)
y_pred = est.predict(test_X)
y_pred
#est_mse = mean_squared_error(y_pred, y_test)
#est_rmse = np.sqrt(est_mse)
#print('Gradient boosting RMSE: %.4f' % est_rmse)

array([2.29172828, 2.29172828, 2.70957951, ..., 2.4196828 , 2.32195982,
       2.22313468])

In [78]:
pd.DataFrame({"id": test_id, "relevance": y_pred}).to_csv('GradientBoosting.csv',index=False)

## Support Vector Machine for Regression.

In [33]:
from sklearn.svm import SVR
clf = SVR(gamma='scale', C=1.0, epsilon=0.2).fit(train_X, train_Y)
y_pred = clf.predict(test_X)
y_pred

array([2.53604977, 2.53604977, 2.5399761 , ..., 2.3120943 , 2.23187932,
       2.32285376])

In [34]:
pd.DataFrame({"id": test_id, "relevance": y_pred}).to_csv('SVM.csv',index=False)

## Decision Tree

In [35]:
# import the regressor 
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 0).fit(train_X, train_Y) 
y_pred = regressor.predict(test_X)
y_pred

array([2.33, 2.33, 2.67, ..., 3.  , 2.33, 1.  ])

In [36]:
pd.DataFrame({"id": test_id, "relevance": y_pred}).to_csv('DT.csv',index=False)